# Поиск похожих сайтов

В этом файле показаны пример использования моделей для составления списка доменов Web-сайтов похожих на заданный.

Word2vec модель (200D) оценивает сходство сайтов использованию их посетителями в одном контексте (несколько упрощенно, сходство указывает на наличие общей аудитории сайтов). Модель содержит 198156 доменов, наиболее посещаемых по данным LiveInternet.

Binary random projection модель (TF-IDF, 512D) описывает сходство сайтов по мешкам слов поисковых запросов. Эта модель содержит 192479 доменов, наиболее посещаемых по данным LiveInternet.

В одной папке с данным файлом должны быть расположены файлы domain_vectors_200D.txt и domain_digest_512D.txt, если это не так, то они загружаются из HDFS.

In [1]:
import pandas as pd
import numpy as np
import os
import IPython

# загрузка word2vec модели и определение функции для работы с ней:

def init_similar_domains_w2v():
    def log(s):        
        IPython.display.clear_output(wait = True)
        print(s)
        
    if not os.path.exists("domain_vectors_200D.txt"):
        log("загрузка из HDFS файла domain_vectors_200D.txt")
        os.system("hadoop fs -get /user/amelnikov/models/domain_vectors_200D.txt domain_vectors_200D.txt")
    
    log("чтение файла domain_vectors_200D.txt")
    w2v = pd.read_csv("domain_vectors_200D.txt", header = None, skiprows = 1, sep = ' ', index_col = False)
    
    log("подготовка структур данных для word2vec модели")
    vects = np.apply_along_axis(lambda x: x/np.linalg.norm(x), 1, np.array(w2v.iloc[:,1:201]))
    words = w2v[0].tolist()
    word_ixs = {w: i for i, w in enumerate(words)}
    del(w2v)

    # Возвращает DataFrame с доменами, упорядоченными по убыванию сходства с заданным доменом domain
    # (метрика сходтсва - косинус угла между векторами доменов):
    def similar_domains_w2v(domain):
        if domain not in word_ixs:
            return pd.DataFrame({'error': ['Домен "%s" не найден' % domain]})
        v = vects[word_ixs[domain]]
        cosa = np.apply_along_axis(lambda x: sum(v*x), 1, vects)
        d = pd.DataFrame({'domain': words, 'cos': cosa}, columns = ['domain', 'cos'])
        return d.sort(['cos'], ascending=[0])
    
    log("")
    return similar_domains_w2v

similar_domains_w2v = init_similar_domains_w2v()


# загрузка random projection модели и определение функции для работы с ней:

def init_similar_domains_sqrp():
    def log(s):        
        IPython.display.clear_output(wait = True)
        print(s)
        
    if not os.path.exists("domain_digest_512D.txt"):
        log("загрузка из HDFS файла domain_digest_512D.txt ...")
        os.system("hadoop fs -get /user/amelnikov/models/domain_digest_512D.txt domain_digest_512D.txt")
        
    log("чтение файла domain_digest_512D.txt")
    dd = pd.read_csv("domain_digest_512D.txt", header = None, sep = ',', index_col = False)

    log("подготовка структур данных для RP модели")
    dm = np.array(dd.iloc[:,1:])
    domains = dd[0].tolist()
    del(dd)    
    domain_ixs = {w: i for i, w in enumerate(domains)}
    
    # Возвращает DataFrame с доменами, упорядоченными по убыванию сходства поисковых запросов с заданным доменом domain
    # (метрика сходтсва - доля одинаковых битов в binary random projection мешков слов поисковых запросов):
    def similar_domains_sqrp(domain):
        if domain not in domain_ixs:
            return pd.DataFrame({'error': ['Домен %s не найден' % domain]})
        v = dm[domain_ixs[domain]]
        dim = 512.0
        bsim = np.apply_along_axis(lambda x: (dim - sum([bin(n).count("1") for n in v^x]))/dim, 1, dm)
        d = pd.DataFrame({'domain': domains, 'bsim': bsim}, columns = ['domain', 'bsim'])
        return d.sort(['bsim'], ascending=[0])  

    log("")
    return similar_domains_sqrp

similar_domains_sqrp = init_similar_domains_sqrp()

## Примеры использования моделей

Домены сайтов следует указывать без www, http и других фрагментов URL !!!

Если пытаемся найти домены похожий на тот, что отсутствует в модели, выдается таблица с ошибкой:

In [2]:
similar_domains_w2v('несуществующий-домен.рф')

,error
0,"Домен ""несуществующий-домен.рф"" не найден"


Задаем домен сайта, для которого нужно получить похожие домены:

In [2]:
url_domain = 'kanobu.ru'

Получаем таблицу доменов, упорядоченную по убыванию сходства с заданным на основе word2vec модели:

In [3]:
d_w2v = similar_domains_w2v(url_domain)

# смотрим первые 10 записей:
d_w2v.head(10)

/home/k.p.osminin/anaconda2/lib/python2.7/site-packages/IPython/kernel/__main__.py:34: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,domain,cos
944,kanobu.ru,1.000000
1010,igromania.ru,0.954960
6354,metagames.ru,0.905270
11033,mgnews.ru,0.901066
615,stopgame.ru,0.884928
1846,shazoo.ru,0.871074
1015,gamebomb.ru,0.866030
11380,game2day.org,0.844209
1338,gamer.ru,0.841205
1918,ag.ru,0.837124


Получаем таблицу доменов, упорядоченную по убыванию сходства поисковых запросов:

In [5]:
d_sqrp = similar_domains_sqrp(url_domain)

# смотрим первые 10 записей:
d_sqrp.head(10)

,domain,bsim
154300,kanobu.ru,1.000000
86732,игры.com,0.785156
62979,gamer-info.com,0.785156
24640,mmoarea.ru,0.781250
62254,gam.kz,0.781250
44130,coolgames.org.ua,0.777344
103590,allagames.ru,0.771484
80716,flashgametop.com,0.771484
171662,igrapro.com,0.769531
189564,noplay.ru,0.767578


Можно построить список доменов, упорядоченный по убыванию средневзвешенного показателя сходства:

In [6]:
# объединяем две таблицы по полю domain:
d_joined = pd.merge(d_w2v, d_sqrp, on="domain")

w_cos = 1.0  # вес сходства по word2vec модели
w_bsim = 1.0 # вес сходства поисковых запросов

#вычисляем средневзвешенный показатель сходтсва:
d_joined['composite'] = (w_cos*d_joined['cos'] + w_bsim*d_joined['bsim'])/(w_cos + w_bsim)

# сортируем таблицу по этому показателю:
d_joined = d_joined.sort(['composite'], ascending=[0]) 

# смотрим первые 20 записей:
d_joined.head(20)

,domain,cos,bsim,composite
0,kanobu.ru,1.000000,1.000000,1.000000
2,metagames.ru,0.905270,0.724609,0.814940
11,gamer-info.com,0.830142,0.785156,0.807649
1,igromania.ru,0.954960,0.626953,0.790957
4,stopgame.ru,0.884928,0.675781,0.780355
9,ag.ru,0.837124,0.710938,0.774031
8,gamer.ru,0.841205,0.666016,0.753611
12,gmbox.ru,0.829511,0.669922,0.749717
13,playground.ru,0.820228,0.664062,0.742145
3,mgnews.ru,0.901066,0.570312,0.735689


## Как сохранить результаты в файл

In [7]:
# чтобы не возникала ошибка UnicodeDecodeError при сохранении результатов в файл:
import sys 
reload(sys)  
sys.setdefaultencoding('utf8')

In [8]:
# сохраняем в файл MS Excel первые 1000 записей:
d_joined[:1000].to_excel("similar_domains.xlsx", index = False)

In [9]:
# сохраняем в CSV файл все записи (кодировка UTF-8, если необходима другая - задать параметр encoding):
d_joined.to_csv("similar_domains.csv", sep = ';', decimal = ',', index = False)